In [1]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine
import getpass  # Para obtener la contraseña sin mostrar la entrada
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df_combined = pd.read_csv('df_combined.csv')

In [ ]:
df_combined

Primero, leemos los datos del archivo CSV y nos aseguramos de insertar las fechas únicas en la tabla dates.

In [ ]:
# Conexión a la base de datos
bd = "financial_analysis"
password = "123456"  # Tu contraseña
connection_string = 'mysql+pymysql://root:' + password + '@localhost/' + bd
engine = create_engine(connection_string)

# Leer el archivo CSV
df_combined = pd.read_csv('df_combined.csv')

# Extraer las fechas únicas del CSV
df_dates = pd.DataFrame(df_combined['date'].unique(), columns=['date'])

# Insertar las fechas en la tabla 'dates'
df_dates.to_sql('dates', con=engine, if_exists='append', index=False)

Después de insertar las fechas, hacemos un merge del archivo CSV con la tabla dates para asignar el id_date correcto en cada fila del CSV.

In [ ]:
# Cargar las fechas desde la tabla 'dates' para obtener el id_date
dates_from_db = pd.read_sql('SELECT * FROM dates', con=engine)

# Asegurarse de que las fechas estén en el mismo formato
df_combined['date'] = pd.to_datetime(df_combined['date']).dt.date
dates_from_db['date'] = pd.to_datetime(dates_from_db['date']).dt.date

# Hacer el merge entre el CSV y la tabla 'dates' usando la columna 'date'
df_merged = pd.merge(df_combined, dates_from_db, how='left', on='date')

# Eliminar la columna 'id_date_x' y renombrar 'id_date_y' a 'id_date'
df_merged.drop(columns=['id_date_x'], inplace=True)
df_merged.rename(columns={'id_date_y': 'id_date'}, inplace=True)

Una vez hecho el merge y corregido el id_date, dividimos los datos entre los activos y los factores económicos y los insertamos en las tablas correspondientes.

In [ ]:
# Dividir el DataFrame en las partes necesarias con el id_date ya corregido
assets_df = df_merged[['id_date', 'price_bitcoin', 'price_gold', 'price_sp500', 'change_bitcoin', 'change_gold']]
economic_factors_df = df_merged[['id_date', 'vix', 'interest_rate', 'cpi', 'inflation']]

# Insertar los datos en las tablas correspondientes
assets_df.to_sql('assets', con=engine, if_exists='append', index=False)
economic_factors_df.to_sql('economic_factors', con=engine, if_exists='append', index=False)

Conexión con MySQL para traer las consultas a Python

In [5]:
# Obtener la contraseña de forma segura
password = getpass.getpass("Introduce la contraseña de MySQL: ")

# Configuración de la base de datos y la conexión
bd = "financial_analysis"  # Nombre de tu base de datos
connection_string = 'mysql+pymysql://root:' + password + '@localhost/' + bd
engine = create_engine(connection_string)

In [6]:
# Consulta 1: Crecimiento anual de Bitcoin, Oro y S&P 500
consulta1 = """
SELECT
    year(d.date) AS year,
    a.price_bitcoin,
    a.price_gold,
    a.price_sp500,
    ROUND( (a.price_bitcoin - LAG(a.price_bitcoin) OVER (ORDER BY year(d.date))) / LAG(a.price_bitcoin) OVER (ORDER BY year(d.date)) * 100, 2) AS bitcoin_growth,
    ROUND( (a.price_gold - LAG(a.price_gold) OVER (ORDER BY year(d.date))) / LAG(a.price_gold) OVER (ORDER BY year(d.date)) * 100, 2) AS gold_growth,
    ROUND( (a.price_sp500 - LAG(a.price_sp500) OVER (ORDER BY year(d.date))) / LAG(a.price_sp500) OVER (ORDER BY year(d.date)) * 100, 2) AS sp500_growth
FROM 
    assets a
JOIN 
    dates d ON a.id_date = d.id_date
WHERE 
    d.date IN (SELECT MAX(d2.date) FROM dates d2 GROUP BY year(d2.date))
ORDER BY 
    year(d.date);
"""
df1 = pd.read_sql(consulta1, con=engine)

In [ ]:
df1

In [ ]:
# Mejorado para consulta 1: Crecimiento anual de Bitcoin, Oro y S&P 500
plt.figure(figsize=(12, 8))

# Gráfico de líneas para cada activo
plt.plot(df1['year'], df1['bitcoin_growth'], label='Crecimiento Bitcoin', marker='o', linestyle='-', color='orange')
plt.plot(df1['year'], df1['gold_growth'], label='Crecimiento Oro', marker='s', linestyle='-', color='gold')
plt.plot(df1['year'], df1['sp500_growth'], label='Crecimiento S&P 500', marker='^', linestyle='-', color='blue')

# Títulos y etiquetas
plt.title('Crecimiento Anual de Bitcoin, Oro y S&P 500 (2015-2024)', fontsize=16, fontweight='bold')
plt.xlabel('Año', fontsize=14)
plt.ylabel('Crecimiento (%)', fontsize=14)

# Leyenda
plt.legend(title='Activos', title_fontsize='13', fontsize='12')

# Añadir etiquetas de datos
for i in range(len(df1)):
    plt.text(df1['year'][i], df1['bitcoin_growth'][i], f"{df1['bitcoin_growth'][i]:.2f}%", fontsize=10, ha='center', color='orange')
    plt.text(df1['year'][i], df1['gold_growth'][i], f"{df1['gold_growth'][i]:.2f}%", fontsize=10, ha='center', color='gold')
    plt.text(df1['year'][i], df1['sp500_growth'][i], f"{df1['sp500_growth'][i]:.2f}%", fontsize=10, ha='center', color='blue')

# Estilo de cuadrícula
plt.grid(True, linestyle='--', alpha=0.6)

# Mostrar gráfico
plt.tight_layout()
plt.show()


In [9]:
# Consulta 2: Promedio y volatilidad mensual de los activos
consulta2 = """
SELECT 
    DATE_FORMAT(d.date, '%%Y-%%m') AS month_year,
    AVG(a.price_bitcoin) AS avg_bitcoin,
    STD(a.price_bitcoin) AS volatility_bitcoin,
    AVG(a.price_gold) AS avg_gold,
    STD(a.price_gold) AS volatility_gold,
    AVG(a.price_sp500) AS avg_sp500,
    STD(a.price_sp500) AS volatility_sp500
FROM 
    assets a
JOIN 
    dates d ON a.id_date = d.id_date
GROUP BY 
    month_year
ORDER BY 
    month_year;
"""

# Ejecutar la consulta y cargar los datos en un DataFrame
df2 = pd.read_sql(consulta2, con=engine)

In [ ]:
# Mostrar el DataFrame resultante
df2

In [11]:
# Consulta 3: Efecto de las tasas de interés en el precio y volatilidad de Bitcoin
consulta3 = """
SELECT
    CASE
        WHEN e.interest_rate <= 2 THEN 'Bajas tasas de interés'
        ELSE 'Altas tasas de interés'
    END AS interest_rate_scenario,
    AVG(a.price_bitcoin) AS avg_bitcoin_price,
    STD(a.price_bitcoin) AS volatility_bitcoin
FROM
    economic_factors e
JOIN
    assets a ON e.id_date = a.id_date
GROUP BY
    interest_rate_scenario;
"""

# Ejecutar la consulta y cargar los datos en un DataFrame
df3 = pd.read_sql(consulta3, con=engine)

In [ ]:
# Mostrar el DataFrame resultante
df3

In [13]:
# Consulta 4: Crecimiento del S&P 500 y su relación con la inflación
consulta4 = """
WITH yearly_data AS (
    SELECT 
        year(d.date) AS year,
        MAX(a.price_sp500) AS price_sp500,
        AVG(e.inflation) AS avg_inflation
    FROM 
        assets a
    JOIN 
        economic_factors e ON a.id_date = e.id_date
    JOIN 
        dates d ON a.id_date = d.id_date
    WHERE 
        d.date IN (SELECT MAX(d2.date) FROM dates d2 GROUP BY year(d2.date))
    GROUP BY 
        year(d.date)
)
SELECT 
    year,
    price_sp500,
    ROUND((price_sp500 - LAG(price_sp500) OVER (ORDER BY year)) / LAG(price_sp500) OVER (ORDER BY year) * 100, 2) AS sp500_growth,
    avg_inflation
FROM 
    yearly_data
ORDER BY 
    year;
"""

# Ejecutar la consulta y cargar los datos en un DataFrame
df4 = pd.read_sql(consulta4, con=engine)

In [ ]:
# Mostrar el DataFrame resultante
df4

In [ ]:
# Mejorado para consulta 4: Crecimiento del S&P 500 y relación con la inflación
fig, ax1 = plt.subplots(figsize=(12, 8))

# Gráfico de líneas para el crecimiento del S&P 500
ax1.plot(df4['year'], df4['sp500_growth'], color='blue', marker='o', label='Crecimiento S&P 500')
ax1.set_xlabel('Año', fontsize=14)
ax1.set_ylabel('Crecimiento del S&P 500 (%)', color='blue', fontsize=14)
ax1.tick_params(axis='y', labelcolor='blue')

# Añadir etiquetas de datos en el gráfico de S&P 500
for i in range(len(df4)):
    ax1.text(df4['year'][i], df4['sp500_growth'][i], f"{df4['sp500_growth'][i]:.2f}%", fontsize=10, ha='center', color='blue')

# Crear un segundo eje Y para la inflación
ax2 = ax1.twinx()
ax2.plot(df4['year'], df4['avg_inflation'], color='red', marker='s', label='Inflación Promedio')
ax2.set_ylabel('Inflación Promedio (%)', color='red', fontsize=14)
ax2.tick_params(axis='y', labelcolor='red')

# Añadir etiquetas de datos en el gráfico de inflación
for i in range(len(df4)):
    ax2.text(df4['year'][i], df4['avg_inflation'][i], f"{df4['avg_inflation'][i]:.2f}%", fontsize=10, ha='center', color='red')

# Título
plt.title('Crecimiento del S&P 500 y su Relación con la Inflación (2015-2024)', fontsize=16, fontweight='bold')

# Leyendas
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

# Cuadrícula
ax1.grid(True, linestyle='--', alpha=0.6)

# Mostrar gráfico
plt.tight_layout()
plt.show()


In [16]:
# Consulta 5: Promedio y volatilidad de Bitcoin en condiciones de alta volatilidad e inflación
consulta5 = """
SELECT
    AVG(a.price_bitcoin) AS avg_bitcoin_price,
    STD(a.price_bitcoin) AS volatility_bitcoin,
    AVG(e.vix) AS avg_vix,
    AVG(e.inflation) AS avg_inflation
FROM
    assets a
JOIN
    economic_factors e ON a.id_date = e.id_date
WHERE
    e.vix > 30 AND e.inflation > 3;
"""

# Ejecutar la consulta y cargar los datos en un DataFrame
df5 = pd.read_sql(consulta5, con=engine)

In [ ]:
# Mostrar el DataFrame resultante
df5

In [1]:
import getpass
import functions as fn

# 1. Leer el CSV
df_combined = fn.load_csv('df_combined.csv')

# 2. Conectar a la base de datos
password = getpass.getpass("Introduce la contraseña de MySQL: ")
engine = fn.create_db_connection(password)

# 3. Insertar fechas y obtener id_date
fn.insert_dates(df_combined, engine)
df_merged = fn.merge_dates(df_combined, engine)

# 4. Dividir e insertar datos en las tablas 'assets' y 'economic_factors'
fn.split_and_insert_data(df_merged, engine)

# 5. Ejecutar y visualizar consulta 1
consulta1 = """
SELECT
    year(d.date) AS year,
    a.price_bitcoin,
    a.price_gold,
    a.price_sp500,
    ROUND( (a.price_bitcoin - LAG(a.price_bitcoin) OVER (ORDER BY year(d.date))) / LAG(a.price_bitcoin) OVER (ORDER BY year(d.date)) * 100, 2) AS bitcoin_growth,
    ROUND( (a.price_gold - LAG(a.price_gold) OVER (ORDER BY year(d.date))) / LAG(a.price_gold) OVER (ORDER BY year(d.date)) * 100, 2) AS gold_growth,
    ROUND( (a.price_sp500 - LAG(a.price_sp500) OVER (ORDER BY year(d.date))) / LAG(a.price_sp500) OVER (ORDER BY year(d.date)) * 100, 2) AS sp500_growth
FROM 
    assets a
JOIN 
    dates d ON a.id_date = d.id_date
WHERE 
    d.date IN (SELECT MAX(d2.date) FROM dates d2 GROUP BY year(d2.date))
ORDER BY 
    year(d.date);
"""
df1 = fn.execute_query(engine, consulta1)
fn.plot_growth_comparison(df1)

# 6. Ejecutar y visualizar consulta 4
consulta4 = """
WITH yearly_data AS (
    SELECT 
        year(d.date) AS year,
        MAX(a.price_sp500) AS price_sp500,
        AVG(e.inflation) AS avg_inflation
    FROM 
        assets a
    JOIN 
        economic_factors e ON a.id_date = e.id_date
    JOIN 
        dates d ON a.id_date = d.id_date
    WHERE 
        d.date IN (SELECT MAX(d2.date) FROM dates d2 GROUP BY year(d2.date))
    GROUP BY 
        year(d.date)
)
SELECT 
    year,
    price_sp500,
    ROUND((price_sp500 - LAG(price_sp500) OVER (ORDER BY year)) / LAG(price_sp500) OVER (ORDER BY year) * 100, 2) AS sp500_growth,
    avg_inflation
FROM 
    yearly_data
ORDER BY 
    year;
"""
df4 = fn.execute_query(engine, consulta4)
fn.plot_sp500_inflation(df4)
